In [37]:
import pandas as pd 
from preprocess_functions import load_data_relevant_cols

In [38]:
raw = load_data_relevant_cols()

/Users/akhiljalan/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2862: DtypeWarning: Columns (4,6,8,10,11,12,17,18,24,29,31,33,35,37,38,39,41,43,45,46,47,49,51,53,54,55,57,58,59,60,61,62,63,64,73,76,79,82,84,86,88,90,92,94,96,97,106,108,114,115,121,123,125,126,127,128,129,134) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/akhiljalan/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2862: DtypeWarning: Columns (6,61,62,63,79,94,96,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,iyear,imonth,iday,country,country_txt,region,region_txt,provstate,city,latitude,...,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ransom,ransomamt
0,1970.0,7.0,2.0,58.0,Dominican Republic,2.0,Central America & Caribbean,NaN,Santo Domingo,18.456792,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
1,1970.0,0.0,0.0,130.0,Mexico,1.0,North America,NaN,Mexico city,19.432608,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1.0,800000.0
2,1970.0,1.0,0.0,160.0,Philippines,5.0,Southeast Asia,Tarlac,Unknown,15.478598,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
3,1970.0,1.0,0.0,78.0,Greece,8.0,Western Europe,Attica,Athens,37.983773,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0,NaN
4,1970.0,1.0,0.0,101.0,Japan,4.0,East Asia,NaN,Fukouka,33.580412,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,0.0,NaN
5,1970.0,1.0,1.0,217.0,United States,1.0,North America,Illinois,Cairo,37.005105,...,0.0,0.0,0.0,1.0,3.0,Minor (likely < $1 million),NaN,NaN,0.0,NaN
6,1970.0,1.0,2.0,218.0,Uruguay,3.0,South America,Montevideo,Montevideo,-34.891151,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
7,1970.0,1.0,2.0,217.0,United States,1.0,North America,California,Oakland,37.805065,...,0.0,0.0,0.0,1.0,3.0,Minor (likely < $1 million),22500.0,Three transformers were damaged.,0.0,NaN
8,1970.0,1.0,2.0,217.0,United States,1.0,North America,Wisconsin,Madison,43.076592,...,0.0,0.0,0.0,1.0,3.0,Minor (likely < $1 million),60000.0,"Basketball courts, weight room, swimming pool,...",0.0,NaN
9,1970.0,1.0,3.0,217.0,United States,1.0,North America,Wisconsin,Madison,43.072950,...,0.0,0.0,0.0,1.0,3.0,Minor (likely < $1 million),NaN,Slight damage,0.0,NaN
